In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import re
import string

url = "https://raw.githubusercontent.com/sahalmaghfud/MLLawanJudi/main/datasetA.csv"
df = pd.read_csv(url)

texts = df['text'].values
labels = df['label'].values



In [2]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=10,shuffle=True)

print(f" training: {len(X_train)}")
print(f"testing: {len(X_test)}")

print(X_test)



 training: 110
testing: 10
['Blibli - online mall Kategori Semua Cari brand, produk, atau seller 0 Masuk Daftar Tambah alamat biar belanja lebih asyik. 08092024/DBE/DES/BLM 08092024/DBE/DES/HTA 08092024/DBE/DES/Fashion 08092024/DBE/DES/Travel Baru widget Lihat Promo Tagihan & Isi Ulang Ekstra Diskon 1Jt Nantikan Promonya Komisi 20% Hemat 1.1Jt Film Gratis BCA Card Tukar Keuangan Bank Official Store Click Collect Ibu Anak Cicilan 0% 2 Jam Sampai Kesehatan Kecantikan Bliblihome 24112024/Long 1/desktop/MKT/blibli-histeria teaser'
 'GoPay Produk Merchant Perusahaan Bantuan Baru! Aplikasi ringan untuk transfer ke mana aja dan bayar apa aja. Download sekarang! Google Play App Store aja, langsung masuk & gratis 100x/bulan Pakai bisa bank sesama GoPay. Syarat Ketentuan berlaku. Call Center: 1500729 customerservice@gopay.co.id Pinjaman Dompet Digital Aman Games Tentang Blog Siaran Pers Pertanyaan Media Karier Lainnya Promo Hubungi Kami Pemberitahuan Privasi Contact Us Gedung Pasaraya Blok M, B,

In [5]:
# Constants
MAX_LENGTH = 700
MAX_VOCAB_SIZE = 10000
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding=padding_type, truncating=trunc_type)

X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding=padding_type, truncating=trunc_type)

vocab_size = len(tokenizer.word_index) + 1

print(f"Vocabulary size: {vocab_size}")


Vocabulary size: 9332


In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(MAX_LENGTH,)),
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=100),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 700, 100)            │         933,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 696, 64)             │          32,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 966,321 (3.69 MB)

 Trainable params: 966,321 (3.69 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
print("X_train_pad shape:", X_train_pad.shape)
print("y_train shape:", y_train.shape)
print("X_test_pad shape:", X_test_pad.shape)
print("y_test shape:", y_test.shape)

X_train_pad shape: (110, 700)
y_train shape: (110,)
X_test_pad shape: (10, 700)
y_test shape: (10,)


In [8]:
history = model.fit(X_train_pad, y_train, epochs=20, validation_data=(X_test_pad, y_test))

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 272ms/step - accuracy: 0.4865 - loss: 0.6918 - val_accuracy: 0.8000 - val_loss: 0.6771
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.6681 - loss: 0.6561 - val_accuracy: 0.9000 - val_loss: 0.6750
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.6446 - loss: 0.6566 - val_accuracy: 1.0000 - val_loss: 0.6656
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.8174 - loss: 0.6080 - val_accuracy: 1.0000 - val_loss: 0.6504
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.7893 - loss: 0.5969 - val_accuracy: 1.0000 - val_loss: 0.6358
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.8262 - loss: 0.5669 - val_accuracy: 1.0000 - val_loss: 0.6223
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 190ms/step - accuracy: 0.8725 - loss: 0.5236 - val_accuracy: 1.0000 - val_loss: 0.6059
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 192ms/step - accuracy: 0.9157 - loss: 0.4742 - val_accuracy: 1.0000 - val_loss:

In [12]:


new_text = ["baim234: HOME Selamat Datang Kakak di Baim234 - Situs Resmi Slot Gacor Anti Rungkad Terbaik Dan Bandar Togel Terpercaya di Indonesia. Member wajib deposit ke rekening yang tertera sekarang di tujuan deposit, jika member deposit ke rekening tujuan deposit yang sebelumnya, maka tidak bisa di proses, sewaktu - waktu tujuan deposit bisa berubah tanpa pemberitahuan. Rupiah Masuk Daftar Lupa Kata Sandi ? Slot Pragmatic Play PG Soft Afb Gaming NEXTSPIN Spade Gaming FASTSPIN Fa Chai OG Slot Advantplay GB Game Playstar JDB NoLimit City HACKSAW Habanero Joker Gaming CQ9 Gaming JILI Microgaming RED TIGER Playtech Slot Casino Pragmatic Play LC OG Casino AFB CASINO WM Casino GD88 SA GAMING Evolution Dream Gaming Asia Gaming Sexy Gaming LG88 WE CASINO PLAYTECH CASINO Live Game BAIM234 LIVE COD GAMING SV388 Cockfight MIKI Gaming WS168 GA28 Sportsbook AFB88 CMD368 SBO SPORT SABA Sports OPUS Sports Lottery 4D Poker We1Poker Poker Q Arcade Spaceman Joker Gaming JILI Fastspin Fa Chai SPRIBE JDB Spade Gaming CQ9 Berita Promosi Daftar Selamat Datang Kakak di Baim234 - Situs Resmi Slot Gacor Anti Rungkad Terbaik Dan Bandar Togel Terpercaya di Indonesia. Member wajib deposit ke rekening yang tertera sekarang di tujuan deposit, jika member deposit ke rekening tujuan deposit yang sebelumnya, maka tidak bisa di proses, sewaktu - waktu tujuan deposit bisa berubah tanpa pemberitahuan. Indonesia Selamat Datang Silakan Login atau Mendaftar Masuk Daftar Lupa Password? Home Event Promosi Berita Informasi Keluaran Bantuan Kontak Kami Region and Language Indonesia Indonesia id English en Rp Game Terpopuler Tampilkan Lainnya SUGAR BANG BANG Legacy Of Kong Maxways MPOPLAY Money Tree Mahjong Wins 3 - Black Scatter Rhapsody of Muertos Mahjong Ways Mpoplay Lucky Twins Wilds FOR THE HORDE Royale House Mahjong Dragon GOLDEN GENIE Gates of Olympus Mahjong Ways 2 Pong Pong Mahjong Jackpots Mahjong Princess MPOPLAY King Arthur Aztec Gold Treasure MAHJONG WAYS 3+ Slot Tampilkan Lainnya Pragmatic Play PG Soft A"]
new_text_seq = tokenizer.texts_to_sequences(new_text)
new_text_pad = pad_sequences(new_text_seq, maxlen=MAX_LENGTH)

prediction = model.predict(new_text_pad)
print(prediction)
print(f"Prediksi: {'Judi Online' if prediction > 0.5 else 'Bukan Judi Online'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[[0.9585825]]
Prediksi: Judi Online


In [10]:
new_text = ["Psikolog Jelaskan Dampak Kecanduan Judi Online buat Anak Remaja Psikolog Jelaskan Dampak Kecanduan Judi Online buat Anak Remaja 0 komentar BAGIKAN Tautan telah disalin MENU detikcom Terpopuler Live TV Kirim Tulisan detikPilkada NEW Kategori Berita detikNews detikFinance detikInet detikHot detikSport Sepakbola detikOto detikTravel detikFood detikHealth Wolipop DetikX 20Detik detikFoto detikEdu detikHikmah detikProperti detikPop NEW Daerah detikJateng detikJatim detikJabar detikSulsel detikSumut detikBali detikSumbagsel detikJogja NEW Layanan Pasang Mata adsmart detikEvent Trans Snow World Trans Studio Flying Over Indonesia berbuatbaik.id ziswafctarsa.id Signature Awards NEW Bingkai.id NEW Detik Network CNN Indonesia CNBC Indonesia Hai Bunda InsertLive Beautynesia Female Daily CXO Media Cancel Yang sedang ramai dicari Loading... Terakhir yang dicari Loading... Home Sekolah Perguruan Tinggi Beasiswa Edutainment Seleksi Masuk PT Detikpedia Foto Video Infografis Indeks Most Popular Bank Soal Point of View detikEdu Sekolah Psikolog Jelaskan Dampak Kecanduan Judi Online buat Anak Remaja Trisna Wulandari - detikEdu Senin, 02 Des 2024 20:00 WIB Muzakki Ramdhan, Egy Octa Suroso, dan Baim Alkatiri di film pendek tentang judi online, Kemenangan Sejati. Foto: Instagram @cerdasberkarakter.kemdikbudri Daftar Isi Dampak Judi Online Ganggu Kesehatan Mental Stres soal Uang Hubungan Merenggang Rentan Kriminalitas Jakarta - Laporan Pusat Pelaporan dan Analisis Transaksi Keuangan (PPATK) menunjukkan tak kurang dari 197.000 anak Indonesia terlibat judi online sepanjang 2024. Anak-anak yang terpapar judi online berada di rentang usia 11-19 tahun. Psikolog Irma Gustiana mengamini bahwa remaja punya rasa ingin tahu yang besar. Karena itu, godaan untuk ingin mencoba judi online terkadang bisa muncul. Namun, ia menegaskan dampak judi online bisa berbuntut panjang. Baca juga: 960 Ribu Pelajar Terlibat Judol, Kemendiktisaintek Bakal Bantu Rehabilitasi Korban Dampak Judi Online Ganggu Kesehatan"]
new_text_seq = tokenizer.texts_to_sequences(new_text)
new_text_pad = pad_sequences(new_text_seq, maxlen=MAX_LENGTH)

prediction = model.predict(new_text_pad)
print(prediction)
print(f"Prediksi: {'Judi Online' if prediction > 0.5 else 'Bukan Judi Online'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[[0.26362738]]
Prediksi: Bukan Judi Online


In [11]:
from google.colab import files
tf.keras.models.save_model(model, 'modelLajuA.keras')
files.download('modelLajuA.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>